In [1]:
import cv2 as cv
import numpy as np
import os
import pandas as pd

In [2]:
def framing(video_path):

    cap = cv.VideoCapture(video_path)
    fps = int(cap.get(5))
    print("fps:", fps)
    if(fps==0):
        print("Not available")
        return 0

    res_fr = []
    while True:
        ret, frame = cap.read()
        if ret:
            resized_frame = cv.resize(frame, (224, 224), interpolation= cv.INTER_LINEAR)
            res_fr.append(resized_frame)
        else:
            break
            
    return res_fr

In [3]:
def keyframeDetection(video_path):
    resized_frames  = framing(video_path)
    
    if(len(resized_frames)==0):
        print("Cannot compare video because of zero fps.")
        return
    
    else:
        video_name = (video_path.split('/')[-1]).split('.')[0]
        keyframePath = 'keyframes/'+video_name
        if not os.path.exists(keyframePath):
            os.makedirs(keyframePath)
        
        diff = []
        for i in range(0,len(resized_frames)-1):
            diff.append(cv.absdiff(resized_frames[i], resized_frames[i+1]))

        mn_all_frames = np.mean(diff)         #This gives mean
        st_d_all_frames = np.std(diff)        #This gives standard deviation
        a = 8                                 #Setting a random value we can modify it to any value 
        ts = mn_all_frames + (a * st_d_all_frames)   # global threshold
        
        local_ts = []
        for i in range(len(diff)):
            mn = np.mean(diff[i])             #Calculating the mean for each frame
            st_d = np.std(diff[i])            #Calculating the standard deviation for each frame
            fr_ts = mn + (5*st_d)             #Finding the threshold values for each frame
            local_ts.append([i,fr_ts])
            

        frame_num = []
        count = 0
        for i,l_tr in(local_ts):
            if l_tr >= ts:                   #Comparing the threshold values to the standard threshold/global threshold values
                cv.imwrite(keyframePath+'/'+str(i)+'.jpg',resized_frames[i]) 
                frame_num.append(i)
                count+=1

        print("Total Number of keyframes saved: {}".format(count))    
        return frame_num, keyframePath

In [7]:
def imageSim(img1_path, img2_path):
    
    img1 = cv.imread(img1_path)
    img2 = cv.imread(img2_path)
    
    finder = cv.xfeatures2d.SIFT_create()
    kp_1, desc_1 = finder.detectAndCompute(img1, None)
    kp_2, desc_2 = finder.detectAndCompute(img2, None)
    
    # FLANN parameters
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary

    flann = cv.FlannBasedMatcher(index_params,search_params)

    matches = flann.knnMatch(desc_1,desc_2,k=2)
    
    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])
    a=len(good)
    percent=(a*100)/len(kp_2)

#     return kp_1, kp_2, desc_1, desc_2, matches
    return percent

In [34]:
k1, k2, d1, d2, m = imageSim("keyframes/test5/35.jpg", "keyframes/test5/37.jpg")
print(len(m))
m

221


[[<DMatch 0000025C901A1BB0>, <DMatch 0000025C901A1A50>],
 [<DMatch 0000025C901A1510>, <DMatch 0000025C901A10B0>],
 [<DMatch 0000025C901A1C90>, <DMatch 0000025C901A1710>],
 [<DMatch 0000025C901A10F0>, <DMatch 0000025C901A1230>],
 [<DMatch 0000025C901A1170>, <DMatch 0000025C901A13F0>],
 [<DMatch 0000025C901A15D0>, <DMatch 0000025C901A1F70>],
 [<DMatch 0000025C901A1B30>, <DMatch 0000025C901A1C10>],
 [<DMatch 0000025C901A1810>, <DMatch 0000025C901A12D0>],
 [<DMatch 0000025C901A1EB0>, <DMatch 0000025C901A1590>],
 [<DMatch 0000025C901A1210>, <DMatch 0000025C901A1850>],
 [<DMatch 0000025C901A1B50>, <DMatch 0000025C901A1E10>],
 [<DMatch 0000025C901A11B0>, <DMatch 0000025C901A16B0>],
 [<DMatch 0000025C901A19B0>, <DMatch 0000025C901A1C30>],
 [<DMatch 0000025C901A1770>, <DMatch 0000025C901A1D50>],
 [<DMatch 0000025C901A1530>, <DMatch 0000025C901A1350>],
 [<DMatch 0000025C901A14D0>, <DMatch 0000025C901A1990>],
 [<DMatch 0000025C901A18B0>, <DMatch 0000025C901A1790>],
 [<DMatch 0000025C901A11D0>, <D

In [21]:
pts = [k1[idx].pt for idx in range(len(k1))]
pts

[(23.62249183654785, 74.35792541503906),
 (25.15420150756836, 71.4920425415039),
 (32.12788772583008, 53.19933319091797),
 (32.12788772583008, 53.19933319091797),
 (32.71330642700195, 64.78168487548828),
 (32.73833465576172, 118.91586303710938),
 (32.912086486816406, 169.42152404785156),
 (33.12593460083008, 119.23184204101562),
 (34.269344329833984, 95.5491714477539),
 (34.269344329833984, 95.5491714477539),
 (35.19859313964844, 115.65826416015625),
 (35.20262908935547, 81.25641632080078),
 (35.2155876159668, 132.3334197998047),
 (35.66029357910156, 138.7112579345703),
 (36.80313491821289, 168.89109802246094),
 (36.894779205322266, 85.0401382446289),
 (36.894779205322266, 85.0401382446289),
 (36.94076156616211, 166.69683837890625),
 (37.024208068847656, 172.93492126464844),
 (37.11299514770508, 92.57828521728516),
 (37.180938720703125, 77.82891082763672),
 (37.180938720703125, 77.82891082763672),
 (37.32421875, 103.19446563720703),
 (37.47154235839844, 96.28612518310547),
 (37.7953338

In [31]:
d1

array([[  0.,   0.,   0., ...,   1.,  78.,  65.],
       [  0.,   0.,   0., ...,   6.,  15.,  26.],
       [  0.,   0.,   0., ...,   1.,   9.,   3.],
       ...,
       [ 21.,  10.,   3., ...,   0.,   0.,   1.],
       [150.,  27.,   0., ...,   4.,  19.,   7.],
       [  5.,   0.,   0., ...,   0.,   0.,   0.]], dtype=float32)

In [43]:
print(len(d1[0]))
print(d1[0][8])
print(d1[0])

128
19.0
[  0.   0.   0.   0.   0.   0.   0.   0.  19.   0.   0.   0.   0.   0.
   4.  29.  23.   0.   0.   0.   0.   1. 141. 104.   0.   0.   0.   0.
   0.   5. 141.  10.   1.   0.   0.   0.   0.   0.   0.   0. 135.   8.
   0.   0.   0.   0.   1.  23. 141.  72.  14.   0.   0.   0.  72. 109.
  17.  47.  35.   0.   0.   1. 141.  27.   1.   0.   0.   0.   0.   0.
   0.   0. 141.   3.   0.   0.   0.   0.   0.  17. 141.  28.   5.   0.
   0.   0.   2.  58.  69.  26.  13.   0.   0.   0.   2.  55.   2.   0.
   0.   0.   0.   0.   0.   0. 141.   0.   0.   0.   0.   0.   0.  34.
 141.   3.   0.   0.   0.   1.  40. 126.  51.   8.   1.   0.   0.   1.
  78.  65.]


In [35]:
print(len(d2[0]))
print(d2[0][8])
print(d2[0])

128
71.0
[  0.   0.   0.   0.   0.   0.   0.   0.  71.   0.   0.   0.   0.   0.
   0.  23. 134.   3.   0.   1.   1.   5.  28. 134.  15.   6.   2.   2.
   2.  15. 131.  41.   1.   0.   0.   0.   0.   0.   0.   0. 134.  13.
   0.   0.   0.   0.   0.  11. 134. 122.  37.   0.   0.   1.   8.  52.
  14.  80. 118.   1.   0.   2.  46.  15.   1.   0.   0.   0.   0.   0.
   0.   0. 124.  11.   0.   0.   0.   0.   0.  28. 134.  75.  23.   1.
   0.   3.  27.  90.   9.  49.  66.   8.   8.  15.  75.  22.   1.   0.
   0.   0.   0.   0.   0.   0.  48.  41.   4.   0.   0.   0.   0.   2.
  59. 134. 110.   0.   0.   0.   4.  10.   3.  60. 134.  32.   7.   5.
  15.   5.]


In [44]:
len(d1)

221

In [45]:
len(d2)

249

In [8]:
def videoSimilarity(video1_path, video2_path):
    v1_keyframe_num, keyframe1_path = keyframeDetection(video1_path)
    v2_keyframe_num, keyframe2_path = keyframeDetection(video2_path)
    
    sim_mat = []
    count=0
    for i in v1_keyframe_num:
        row = []
        p1 = keyframe1_path+'/'+str(i)+'.jpg'
        for j in v2_keyframe_num:  
            p2 = keyframe2_path+'/'+str(j)+'.jpg'
            sim = imageSim(p1, p2)
            row.append(sim)

        count = count+1
        print("row",count, end='\r')
        sim_mat.append(row)
        
    sim_df = pd.DataFrame(sim_mat)
#     sim_percent = ((sim_df.max(axis=1).sum())/len(sim_df))
    return sim_df

### BlackBorder & Speeder Videos

In [6]:
df = videoSimilarity("test-videos/test5.mp4", "test-videos/test6.mp4")
df

fps: 30
Total Number of keyframes saved: 31
fps: 30
Total Number of keyframes saved: 14


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,10.062893,9.230769,15.546218,8.550186,14.529915,6.644518,2.173913,3.267974,1.408451,3.571429,2.985075,3.252033,5.851064,4.964539
1,16.666667,11.153846,11.764706,9.293680,11.111111,5.315615,3.623188,2.941176,0.938967,2.380952,1.791045,6.504065,2.659574,4.255319
2,19.182390,10.769231,13.025210,8.550186,8.119658,4.983389,3.260870,2.941176,0.938967,1.785714,2.089552,4.065041,2.127660,7.801418
3,36.792453,14.615385,17.226891,10.408922,9.829060,7.308970,3.623188,3.594771,1.408451,4.761905,3.880597,8.130081,3.191489,6.382979
4,20.125786,17.692308,9.663866,10.780669,10.256410,7.308970,2.898551,4.901961,1.408451,3.571429,2.686567,4.878049,1.595745,7.092199
5,12.264151,33.846154,17.226891,8.550186,10.256410,6.312292,2.173913,3.594771,2.347418,2.380952,2.388060,3.252033,1.063830,4.255319
6,9.119497,15.384615,42.436975,9.293680,11.965812,4.318937,4.710145,3.267974,1.408451,1.190476,2.089552,4.878049,5.319149,2.836879
7,6.918239,7.307692,12.184874,32.342007,19.658120,7.973422,4.710145,4.248366,3.286385,3.571429,2.388060,1.626016,6.382979,6.382979
8,5.974843,5.384615,13.865546,15.613383,27.777778,11.960133,7.246377,4.575163,2.816901,2.380952,1.791045,2.439024,5.319149,6.382979
9,2.830189,5.384615,11.344538,11.152416,19.658120,30.564784,11.594203,7.516340,1.408451,2.380952,3.283582,4.878049,2.659574,3.546099


In [25]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
max_df.sum()/len(max_df)

38.307631032203716

In [8]:
print(df.max(axis=1).sum()/len(df))

26.036791077561567


### Videos with Jumbled Frames

In [26]:
df = videoSimilarity("test-videos/test7.mp4", "test-videos/test8.mp4")
df

fps: 30
Total Number of keyframes saved: 70
fps: 25
Total Number of keyframes saved: 56


,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,11.111111,1.351351,1.960784,2.542373,2.702703,1.265823,1.990050,1.626016,1.680672,0.858369,...,7.808564,7.073171,4.952830,8.040201,8.000000,6.648199,4.722222,3.846154,3.278689,1.990050
1,14.285714,1.013514,3.137255,3.389831,2.702703,2.109705,3.980100,1.626016,2.521008,3.433476,...,9.571788,7.804878,8.018868,9.296482,6.588235,8.310249,5.000000,5.029586,3.278689,1.492537
2,14.285714,1.689189,2.745098,2.118644,2.316602,3.375527,3.980100,2.032520,2.100840,0.858369,...,9.068010,6.829268,7.547170,9.296482,6.117647,7.479224,3.888889,5.325444,3.005464,3.482587
3,4.761905,2.364865,4.313725,2.966102,3.474903,2.531646,4.477612,1.219512,2.521008,2.575107,...,9.571788,9.268293,7.547170,10.804020,6.823529,7.479224,4.722222,4.142012,4.098361,1.990050
4,22.222222,1.351351,3.137255,4.237288,4.247104,2.953586,2.985075,2.032520,1.260504,1.287554,...,10.075567,7.804878,8.962264,9.296482,7.058824,9.695291,6.388889,2.958580,4.371585,2.985075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,9.523810,1.351351,0.392157,2.966102,3.474903,2.531646,3.482587,2.439024,1.680672,1.716738,...,3.778338,4.390244,4.481132,3.768844,3.294118,3.878116,3.888889,3.550296,1.366120,1.990050
66,7.936508,1.689189,1.176471,3.813559,2.702703,2.109705,2.985075,1.626016,2.100840,0.858369,...,6.045340,4.878049,4.952830,4.271357,3.058824,4.986150,4.722222,2.662722,2.732240,0.995025
67,4.761905,1.351351,2.352941,1.694915,3.088803,2.531646,2.487562,2.845528,2.521008,2.145923,...,7.052897,6.829268,4.952830,5.527638,5.882353,5.540166,5.000000,2.662722,3.278689,2.985075
68,12.698413,1.351351,2.352941,1.694915,1.930502,2.531646,1.990050,2.439024,1.680672,0.858369,...,7.556675,6.097561,4.952830,6.783920,7.294118,6.648199,4.722222,4.142012,4.371585,2.487562


In [27]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
max_df.sum()/len(max_df)

67.49689185600948

In [10]:
print(df.max(axis=1).sum()/len(df))

58.683826629416835


### -------------------------------------

In [28]:
df = videoSimilarity("test-videos/test9.mp4", "test-videos/test10.mp4")
df

fps: 30
Total Number of keyframes saved: 16
fps: 25
Total Number of keyframes saved: 16


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,77.910448,4.137931,17.897727,2.857143,2.597403,1.960784,2.194357,2.487562,1.030928,1.149425,1.632653,0.865801,0.925926,2.072539,1.020408,0.549451
1,3.880597,72.413793,4.261364,8.571429,0.000000,0.653595,0.626959,9.452736,0.000000,0.000000,0.816327,1.298701,1.388889,1.554404,0.680272,0.549451
2,16.716418,3.103448,75.284091,3.174603,2.922078,3.267974,0.000000,0.995025,1.030928,1.149425,1.632653,0.865801,2.314815,3.108808,0.340136,1.098901
3,2.388060,8.965517,3.125000,75.873016,0.974026,3.267974,0.626959,2.985075,1.546392,1.436782,3.673469,2.164502,0.925926,1.554404,1.020408,1.098901
4,1.492537,5.862069,0.852273,0.952381,0.974026,0.653595,0.940439,77.611940,1.030928,0.862069,1.224490,1.731602,2.777778,1.036269,0.340136,1.098901
5,2.686567,0.689655,1.420455,1.269841,0.324675,4.575163,4.702194,3.980100,74.742268,60.344828,50.612245,40.692641,25.925926,18.652850,4.761905,9.890110
6,2.089552,0.689655,1.420455,1.269841,1.623377,4.575163,3.761755,4.477612,57.989691,78.160920,60.816327,46.753247,29.629630,20.207254,4.081633,8.241758
7,1.791045,1.034483,1.420455,1.269841,0.974026,5.228758,4.075235,2.985075,35.051546,45.402299,80.000000,58.441558,34.259259,24.870466,4.421769,7.692308
8,1.791045,0.344828,0.568182,0.634921,1.298701,6.535948,3.448276,3.980100,25.515464,33.333333,55.102041,83.982684,43.981481,34.196891,6.122449,7.142857
9,1.194030,0.344828,1.704545,3.809524,4.870130,2.614379,1.567398,1.492537,13.917526,16.954023,30.612245,39.826840,75.925926,48.704663,7.482993,13.186813


In [29]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
max_df.sum()/len(max_df)

77.24731107998174

In [24]:
print(df.max(axis=1).sum()/len(df))

72.55671677484551


### -------------------------------------

In [30]:
df = videoSimilarity("test-videos/test11.mp4", "test-videos/test12.mp4")
df

fps: 25
Total Number of keyframes saved: 32
fps: 30
Total Number of keyframes saved: 45


,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,1.098901,2.463054,3.982301,3.108808,3.017241,1.492537,0.342466,0.724638,2.583026,1.063830,...,2.030457,1.707317,1.412429,1.423488,0.327869,1.282051,4.214559,0.917431,1.424501,0.772201
1,2.197802,0.492611,0.442478,4.145078,1.293103,0.746269,0.342466,0.362319,1.107011,0.000000,...,1.015228,0.975610,0.282486,0.711744,0.655738,0.256410,0.383142,0.688073,0.284900,0.000000
2,1.098901,2.955665,3.097345,1.554404,6.034483,1.119403,3.767123,2.173913,2.214022,0.709220,...,0.761421,0.731707,1.129944,2.135231,1.967213,1.282051,0.766284,0.688073,0.569801,0.772201
3,0.549451,1.477833,2.212389,1.554404,3.448276,1.492537,0.684932,1.811594,1.107011,1.063830,...,4.822335,6.097561,7.344633,3.558719,7.868852,8.974359,3.448276,5.963303,5.982906,0.386100
4,0.000000,0.492611,2.212389,1.554404,1.293103,1.119403,0.342466,0.000000,1.845018,0.709220,...,1.522843,0.487805,1.412429,3.558719,1.639344,1.538462,1.149425,1.376147,2.849003,0.000000
5,1.098901,3.940887,2.212389,2.072539,0.862069,1.492537,1.712329,1.086957,1.845018,1.418440,...,7.614213,5.609756,7.627119,4.626335,5.573770,4.871795,5.747126,3.669725,5.413105,0.386100
6,1.098901,1.970443,2.212389,1.554404,1.293103,1.492537,0.342466,1.449275,0.738007,1.418440,...,1.522843,1.219512,1.412429,3.558719,2.622951,2.564103,1.915709,0.229358,1.139601,0.386100
7,2.747253,1.477833,3.982301,2.072539,1.724138,0.746269,1.712329,1.449275,1.476015,1.418440,...,9.644670,9.024390,13.276836,13.167260,7.868852,6.153846,11.111111,6.192661,7.407407,1.544402
8,1.098901,1.970443,2.212389,2.590674,0.431034,0.746269,0.684932,1.086957,0.369004,0.354610,...,7.614213,7.317073,8.757062,11.032028,15.409836,5.897436,13.409962,6.422018,9.686610,0.772201
9,1.098901,1.970443,2.654867,2.590674,2.586207,1.119403,2.054795,0.724638,2.583026,2.127660,...,26.142132,18.048780,18.079096,9.964413,15.737705,18.205128,15.708812,13.532110,12.535613,0.772201


In [31]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
max_df.sum()/len(max_df)

71.78798016381957

In [22]:
print(df.max(axis=1).sum()/len(df))

68.54149114384057


### -------------------------------------

In [10]:
df = videoSimilarity("test-videos/new-test2-og.mp4", "test-videos/new-test2-dup.mp4")
df

fps: 30
Total Number of keyframes saved: 68
fps: 30
Total Number of keyframes saved: 52


,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,1.941748,2.380952,0.000000,1.834862,0.444444,0.358423,0.000000,0.352113,0.350877,1.444043,...,1.785714,1.219512,4.6875,7.407407,36.363636,23.782235,15.691489,0.909091,3.389831,0.381679
1,4.854369,3.809524,0.483092,2.752294,0.888889,1.075269,1.127820,0.704225,1.403509,1.444043,...,0.892857,1.219512,7.8125,7.407407,31.818182,55.587393,42.819149,2.727273,0.847458,0.763359
2,3.883495,1.904762,0.000000,1.376147,0.000000,1.075269,1.127820,1.056338,2.105263,1.444043,...,1.785714,0.000000,7.8125,0.000000,4.545455,36.962751,51.329787,1.818182,0.847458,0.381679
3,3.883495,2.380952,0.483092,1.376147,1.777778,1.433692,0.751880,0.352113,1.754386,1.805054,...,1.785714,3.658537,10.9375,7.407407,31.818182,0.286533,0.000000,2.727273,4.237288,61.450382
4,0.970874,1.428571,0.483092,0.458716,0.000000,0.000000,0.000000,0.352113,0.350877,0.000000,...,1.785714,2.439024,1.5625,0.000000,18.181818,0.286533,0.000000,0.000000,0.000000,0.381679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,2.912621,2.380952,0.483092,2.293578,0.444444,0.716846,0.751880,0.352113,2.105263,0.722022,...,2.678571,6.097561,7.8125,3.703704,59.090909,0.573066,0.265957,4.545455,0.847458,1.145038
64,5.825243,3.333333,1.932367,1.834862,0.888889,0.358423,1.503759,1.056338,0.701754,0.722022,...,69.642857,0.000000,3.1250,0.000000,13.636364,0.000000,0.000000,0.000000,1.694915,0.381679
65,1.941748,0.476190,0.000000,0.917431,0.888889,0.716846,0.751880,1.408451,0.350877,1.444043,...,3.571429,71.951220,3.1250,3.703704,31.818182,0.000000,0.265957,0.909091,0.847458,0.763359
66,2.912621,1.428571,0.000000,2.293578,0.888889,1.433692,0.751880,1.056338,1.403509,1.083032,...,6.250000,3.658537,81.2500,3.703704,50.000000,0.000000,0.531915,0.000000,1.694915,0.381679


In [11]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

71.96883003029274


### Dissimilar Videos

In [32]:
df = videoSimilarity("test-videos/test9.mp4", "test-videos/test11.mp4")
df

fps: 30
Total Number of keyframes saved: 16
fps: 25
Total Number of keyframes saved: 32


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.414938,1.941748,0.000000,0.533333,2.205882,1.766784,6.349206,1.880878,2.752294,1.176471,...,0.000000,0.740741,1.086957,3.773585,1.809955,1.554404,2.347418,0.404858,5.194805,2.439024
1,1.659751,2.912621,0.898204,1.866667,4.411765,1.413428,5.555556,1.880878,4.128440,0.235294,...,0.000000,0.740741,0.543478,1.415094,1.357466,3.108808,2.347418,0.404858,3.896104,3.658537
2,0.829876,4.854369,0.299401,0.800000,0.000000,1.413428,4.761905,0.313480,1.834862,1.176471,...,0.408163,0.370370,2.173913,1.886792,2.262443,2.072539,1.408451,0.809717,6.493506,7.317073
3,2.074689,3.883495,0.898204,2.133333,5.882353,2.826855,4.761905,1.253918,0.917431,0.235294,...,0.816327,1.481481,2.717391,1.415094,2.714932,2.590674,5.164319,0.404858,7.792208,8.536585
4,2.074689,0.970874,0.000000,1.600000,3.676471,1.413428,4.761905,1.253918,0.917431,0.941176,...,0.000000,0.370370,0.543478,1.415094,2.714932,1.554404,0.938967,1.214575,2.597403,0.000000
5,1.244813,7.766990,1.197605,0.533333,5.147059,1.413428,3.968254,1.253918,1.376147,0.941176,...,0.000000,0.740741,0.543478,1.886792,3.167421,2.590674,4.225352,1.619433,2.597403,2.439024
6,1.244813,3.883495,0.898204,0.800000,4.411765,0.706714,3.174603,0.313480,1.376147,0.705882,...,0.000000,0.370370,1.630435,1.886792,1.809955,1.554404,3.755869,2.429150,3.896104,3.658537
7,1.244813,6.796117,0.299401,0.533333,4.411765,1.413428,1.587302,0.313480,1.834862,1.411765,...,0.000000,0.370370,1.630435,2.830189,3.167421,1.554404,2.816901,1.214575,2.597403,2.439024
8,2.074689,2.912621,0.299401,1.333333,6.617647,1.060071,3.174603,0.313480,1.376147,0.941176,...,0.816327,0.000000,1.086957,2.830189,2.714932,2.590674,2.816901,2.024291,3.896104,9.756098
9,5.394191,2.912621,0.299401,2.133333,6.617647,2.120141,1.587302,0.000000,1.376147,2.117647,...,0.408163,0.370370,2.173913,2.358491,6.334842,7.772021,2.347418,2.024291,2.597403,6.097561


In [36]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

6.865378122628382


In [7]:
print(df.max(axis=1).sum()/len(df))

6.865378122628382


### -------------------------------------

In [37]:
df = videoSimilarity("test-videos/test4.mp4", "test-videos/test5.mp4")
df

fps: 30
Total Number of keyframes saved: 8
fps: 30
Total Number of keyframes saved: 31


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.000000,2.409639,0.438596,2.643172,1.507538,0.0,5.294118,0.0,0.0,0.000000,...,0.000000,0.000000,3.378378,0.000000,0.0,0.000000,0.000000,1.388889,1.342282,1.307190
1,0.452489,0.000000,0.000000,0.000000,0.000000,0.0,0.588235,0.0,0.0,0.555556,...,0.370370,0.000000,0.000000,0.000000,0.0,0.719424,0.793651,0.694444,0.671141,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.653595
3,0.000000,0.000000,0.000000,0.440529,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.694444,0.671141,0.000000
4,0.000000,1.606426,0.000000,1.762115,0.000000,0.0,1.176471,0.0,0.0,1.666667,...,1.111111,0.000000,1.351351,1.886792,0.0,0.000000,0.793651,1.388889,3.355705,1.307190
5,0.000000,0.803213,0.438596,1.321586,0.000000,0.0,2.941176,0.0,0.0,0.555556,...,0.000000,0.000000,2.027027,0.000000,0.0,0.000000,0.793651,2.083333,1.342282,1.307190
6,0.452489,0.803213,0.000000,0.881057,1.005025,0.0,1.176471,0.0,0.0,0.000000,...,0.000000,0.000000,1.351351,0.943396,0.0,0.000000,1.587302,2.083333,1.342282,0.653595
7,0.000000,0.000000,0.000000,0.000000,0.502513,0.0,0.000000,0.0,0.0,1.111111,...,0.000000,0.884956,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [38]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

2.166622178157413


In [18]:
print(df.max(axis=1).sum()/len(df))

2.166622178157413


### Same Videos

In [7]:
df = videoSimilarity("test-videos/test9.mp4", "test-videos/test9.mp4")
df

fps: 30
Total Number of keyframes saved: 16
fps: 30
Total Number of keyframes saved: 16


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,100.000000,4.705882,16.253444,3.313253,2.512563,2.754821,2.616279,1.960784,1.315789,1.546392,1.648352,1.030928,1.648352,0.0,2.760736,3.246753
1,3.519062,100.000000,4.683196,7.530120,9.547739,0.550964,0.000000,0.784314,0.877193,2.577320,1.648352,0.687285,0.000000,0.0,0.306748,1.298701
2,18.181818,5.098039,100.000000,3.915663,1.005025,2.203857,1.744186,1.176471,0.877193,2.577320,2.197802,0.343643,1.648352,0.0,3.067485,1.948052
3,3.225806,9.803922,4.407713,100.000000,1.507538,1.652893,0.872093,3.137255,2.192982,3.608247,2.747253,1.718213,2.197802,0.0,0.306748,1.298701
4,2.052786,5.882353,0.826446,0.903614,100.000000,1.377410,2.034884,1.960784,2.192982,2.061856,1.648352,1.030928,2.197802,0.0,0.613497,0.649351
5,2.639296,2.352941,1.101928,0.903614,3.015075,100.000000,65.988372,54.509804,43.421053,29.896907,19.230769,4.467354,7.142857,0.0,0.613497,1.948052
6,2.639296,1.960784,1.652893,1.506024,2.010050,62.809917,100.000000,67.843137,51.754386,34.020619,21.978022,3.780069,6.043956,0.0,0.920245,1.948052
7,2.639296,2.352941,1.652893,1.204819,1.507538,37.190083,50.000000,100.000000,65.789474,39.690722,25.274725,5.154639,9.340659,0.0,0.920245,1.298701
8,1.466276,1.568627,1.101928,0.602410,1.507538,26.997245,34.011628,58.823529,100.000000,48.453608,32.417582,6.872852,9.340659,0.0,1.226994,2.597403
9,1.466276,1.176471,3.030303,3.915663,1.507538,14.049587,18.023256,29.019608,42.105263,100.000000,52.197802,8.591065,13.186813,0.0,1.533742,1.948052


In [8]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

100.0


In [6]:
df = videoSimilarity("test-videos/test-video1-og.mp4", "test-videos/test-video1-BlackBorder.mp4")
df

fps: 25
Total Number of keyframes saved: 20
fps: 30
Total Number of keyframes saved: 32


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,27.692308,45.967742,21.705426,23.622047,5.185185,9.302326,9.090909,14.285714,10.714286,11.711712,...,15.503876,13.559322,9.589041,10.563380,8.527132,9.701493,17.037037,13.986014,11.403509,10.317460
1,19.230769,28.225806,51.937984,29.921260,5.185185,7.751938,13.223140,10.084034,9.821429,11.711712,...,9.302326,12.711864,10.958904,7.746479,13.178295,14.179104,11.851852,11.188811,13.157895,11.904762
2,6.153846,5.645161,5.426357,7.874016,24.444444,35.658915,49.586777,38.655462,8.035714,9.909910,...,9.302326,7.627119,10.273973,7.746479,11.627907,11.940299,8.148148,6.293706,8.771930,6.349206
3,7.692308,8.870968,5.426357,11.811024,19.259259,26.356589,38.842975,54.621849,8.928571,11.711712,...,11.627907,8.474576,13.013699,11.971831,13.178295,14.925373,9.629630,8.391608,10.526316,7.142857
4,8.461538,13.709677,6.976744,9.448819,8.888889,6.201550,12.396694,11.764706,21.428571,26.126126,...,7.751938,5.084746,4.109589,7.042254,6.201550,7.462687,5.185185,5.594406,7.017544,3.174603
5,7.692308,10.483871,4.651163,8.661417,7.407407,6.976744,9.917355,10.084034,11.607143,18.018018,...,8.527132,7.627119,7.534247,6.338028,6.976744,7.462687,4.444444,4.195804,7.894737,4.761905
6,6.153846,6.451613,3.100775,5.511811,10.370370,10.077519,9.917355,10.084034,5.357143,7.207207,...,9.302326,10.169492,10.273973,9.154930,12.403101,11.194030,10.370370,10.489510,11.403509,10.317460
7,5.384615,4.838710,3.875969,7.086614,11.851852,11.627907,14.049587,15.126050,7.142857,4.504505,...,14.728682,13.559322,11.643836,9.859155,16.279070,11.194030,13.333333,13.286713,14.912281,8.730159
8,4.615385,4.838710,3.875969,6.299213,8.148148,6.976744,14.049587,14.285714,5.357143,9.909910,...,11.627907,8.474576,8.904110,10.563380,16.279070,17.910448,11.111111,10.489510,10.526316,9.523810
9,9.230769,5.645161,10.077519,4.724409,8.148148,6.976744,9.917355,6.722689,7.142857,8.108108,...,12.403101,5.932203,11.643836,7.042254,10.077519,16.417910,13.333333,13.286713,12.280702,10.317460


In [7]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

47.763848740779046


In [8]:
df = videoSimilarity("test-videos/test-video1-og.mp4", "test-videos/test-video1-increased-speed.mp4")
df

fps: 25
Total Number of keyframes saved: 20
fps: 30
Total Number of keyframes saved: 21


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,84.353741,43.571429,12.8,12.244898,18.439716,15.942029,12.408759,13.380282,10.687023,10.948905,...,11.409396,10.218978,12.056738,17.482517,15.322581,16.030534,12.676056,16.901408,16.788321,16.233766
1,43.537415,95.000000,12.0,14.285714,21.985816,15.217391,14.598540,11.267606,9.923664,13.138686,...,12.751678,8.759124,9.929078,8.391608,17.741935,12.977099,12.676056,11.267606,13.138686,14.285714
2,11.564626,10.714286,86.4,47.619048,17.021277,15.217391,13.868613,11.267606,15.267176,10.218978,...,10.738255,12.408759,10.638298,13.286713,16.935484,15.267176,18.309859,13.380282,13.868613,12.987013
3,10.204082,15.714286,55.2,82.993197,19.858156,21.014493,13.138686,10.563380,17.557252,8.759124,...,13.422819,12.408759,12.765957,14.685315,20.967742,16.793893,19.718310,12.676056,12.408759,14.285714
4,16.326531,19.285714,18.4,19.047619,91.489362,47.826087,11.678832,9.859155,8.396947,10.218978,...,8.053691,9.489051,5.673759,8.391608,12.903226,9.160305,9.154930,9.859155,13.868613,11.688312
5,14.285714,17.142857,16.8,18.367347,48.226950,85.507246,8.029197,13.380282,8.396947,10.218978,...,7.382550,8.759124,7.092199,8.391608,12.903226,9.160305,7.746479,9.154930,10.218978,9.740260
6,10.884354,10.000000,17.6,10.884354,9.929078,10.869565,89.781022,43.661972,31.297710,19.708029,...,12.751678,14.598540,12.765957,14.685315,13.709677,14.503817,16.197183,12.676056,16.058394,11.038961
7,11.564626,9.285714,16.8,10.884354,10.638298,12.318841,43.795620,90.845070,42.748092,24.817518,...,18.791946,19.708029,17.021277,13.986014,18.548387,18.320611,14.788732,13.380282,21.167883,17.532468
8,9.523810,9.285714,23.2,15.646259,11.347518,13.043478,29.197080,36.619718,87.022901,37.956204,...,19.463087,22.627737,17.730496,13.286713,18.548387,13.740458,16.197183,11.971831,16.788321,14.935065
9,14.285714,10.714286,13.6,6.122449,9.929078,13.768116,19.708029,26.056338,41.221374,84.671533,...,22.818792,24.087591,17.730496,19.580420,19.354839,17.557252,20.422535,16.901408,16.058394,13.636364


In [9]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

87.50415247243157


In [10]:
df = videoSimilarity("test-videos/test-video1-og.mp4", "test-videos/test-video1-with-watermark.mp4")
df

fps: 25
Total Number of keyframes saved: 20
fps: 30
Total Number of keyframes saved: 30


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,39.890710,73.006135,35.668790,26.744186,9.933775,10.077519,7.971014,13.605442,15.853659,14.465409,...,14.649682,12.857143,15.068493,8.805031,9.638554,13.068182,14.465409,15.882353,16.071429,14.285714
1,30.054645,35.582822,77.070064,34.883721,9.933775,11.627907,9.420290,14.285714,14.634146,16.352201,...,8.280255,12.142857,11.643836,11.320755,8.433735,14.772727,10.062893,10.588235,13.095238,10.119048
2,6.010929,8.588957,8.280255,9.883721,35.099338,51.162791,74.637681,46.938776,11.585366,11.320755,...,11.464968,12.857143,11.643836,16.981132,17.469880,11.931818,8.805031,8.235294,11.309524,10.119048
3,8.743169,10.429448,11.464968,9.302326,31.125828,39.534884,51.449275,78.911565,16.463415,14.465409,...,14.649682,14.285714,16.438356,17.610063,16.867470,12.500000,7.547170,7.058824,10.714286,9.523810
4,12.021858,13.496933,15.923567,13.372093,8.609272,12.403101,13.043478,18.367347,40.243902,77.358491,...,5.732484,5.714286,8.219178,8.805031,9.036145,8.522727,8.176101,7.058824,9.523810,8.928571
5,11.475410,12.883436,13.375796,11.046512,7.947020,11.627907,13.043478,15.646259,32.317073,40.880503,...,5.095541,7.142857,6.849315,7.547170,9.638554,7.386364,6.289308,7.058824,8.333333,5.357143
6,5.464481,7.361963,7.643312,6.395349,13.907285,11.627907,10.869565,8.843537,9.146341,6.289308,...,12.101911,10.000000,13.698630,12.578616,12.048193,13.068182,13.207547,10.588235,8.333333,17.857143
7,8.196721,7.975460,8.917197,7.558140,14.569536,13.953488,11.594203,11.564626,8.536585,9.433962,...,12.738854,15.000000,16.438356,15.094340,16.265060,14.772727,13.207547,15.882353,13.095238,12.500000
8,9.836066,6.134969,6.369427,5.813953,13.907285,17.054264,13.768116,16.326531,7.317073,7.547170,...,11.464968,14.285714,13.013699,15.723270,21.084337,19.318182,10.062893,11.764706,11.904762,14.285714
9,8.743169,11.042945,9.554140,4.651163,8.609272,10.077519,7.246377,9.523810,7.926829,6.289308,...,19.108280,14.285714,14.383562,15.723270,18.072289,20.454545,14.465409,14.117647,11.309524,16.071429


In [11]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

74.17026083714032


In [12]:
df = videoSimilarity("test-videos/test-video1-og.mp4", "test-videos/test-video1-BorderAndWatermark.mp4")
df

fps: 25
Total Number of keyframes saved: 20
fps: 30
Total Number of keyframes saved: 32


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,26.950355,42.18750,22.962963,23.076923,7.432432,6.617647,8.130081,11.023622,11.764706,8.474576,...,14.705882,12.698413,9.740260,9.271523,10.000000,10.370370,15.540541,12.318841,9.302326,8.461538
1,19.858156,25.00000,48.148148,26.923077,6.756757,8.823529,8.130081,8.661417,10.924370,9.322034,...,8.823529,11.904762,10.389610,7.947020,13.571429,13.333333,9.459459,11.594203,8.527132,10.769231
2,4.964539,7.03125,6.666667,6.153846,25.000000,34.558824,50.406504,32.283465,6.722689,7.627119,...,8.823529,7.142857,11.038961,7.284768,12.142857,10.370370,6.081081,7.971014,6.976744,5.384615
3,7.092199,9.37500,8.148148,12.307692,19.594595,19.852941,37.398374,46.456693,8.403361,7.627119,...,11.029412,7.936508,13.636364,13.245033,15.000000,11.851852,7.432432,7.246377,8.527132,8.461538
4,9.929078,7.81250,7.407407,10.769231,6.756757,5.147059,8.943089,14.173228,20.168067,24.576271,...,7.352941,4.761905,5.194805,6.622517,7.142857,7.407407,4.054054,7.971014,5.426357,3.076923
5,9.929078,8.59375,6.666667,9.230769,6.081081,7.352941,8.130081,11.023622,11.764706,15.254237,...,8.088235,7.936508,7.142857,6.622517,7.142857,8.888889,3.378378,7.246377,5.426357,4.615385
6,9.929078,6.25000,2.962963,3.076923,6.081081,8.088235,9.756098,6.299213,2.521008,5.084746,...,8.823529,9.523810,9.740260,9.271523,12.142857,11.851852,8.108108,8.695652,8.527132,9.230769
7,7.801418,3.90625,4.444444,9.230769,8.783784,8.823529,8.943089,12.598425,7.563025,2.542373,...,13.970588,11.904762,12.337662,9.271523,17.857143,10.370370,8.783784,12.318841,11.627907,9.230769
8,5.673759,7.03125,4.444444,5.384615,9.459459,9.558824,10.569106,12.598425,5.042017,5.932203,...,11.029412,7.142857,9.740260,10.596026,17.857143,15.555556,9.459459,8.695652,8.527132,12.307692
9,11.347518,11.71875,6.666667,5.384615,7.432432,8.088235,7.317073,6.299213,9.243697,7.627119,...,11.764706,6.349206,12.337662,7.947020,11.428571,17.037037,12.837838,10.144928,10.852713,11.538462


In [13]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

44.62201090319426


In [14]:
df = videoSimilarity("test-videos/test-video2-og.mp4", "test-videos/test-video2-BlackBorder.mp4")
df

fps: 25
Total Number of keyframes saved: 92
fps: 30
Total Number of keyframes saved: 124


,0,1,2,3,4,5,6,7,8,9,...,114,115,116,117,118,119,120,121,122,123
0,26.086957,30.909091,26.923077,17.857143,13.846154,26.923077,35.849057,24.489796,23.913043,16.0,...,13.207547,10.169492,10.000000,17.073171,14.285714,10.416667,15.0,17.5,7.843137,8.888889
1,19.565217,20.000000,17.307692,8.928571,13.846154,19.230769,22.641509,30.612245,26.086957,18.0,...,15.094340,10.169492,11.666667,17.073171,17.142857,12.500000,20.0,20.0,9.803922,15.555556
2,13.043478,27.272727,26.923077,21.428571,16.923077,19.230769,22.641509,18.367347,21.739130,30.0,...,15.094340,20.338983,8.333333,14.634146,28.571429,12.500000,25.0,17.5,17.647059,13.333333
3,8.695652,14.545455,19.230769,12.500000,21.538462,19.230769,13.207547,14.285714,17.391304,28.0,...,22.641509,28.813559,10.000000,14.634146,22.857143,16.666667,25.0,20.0,11.764706,13.333333
4,19.565217,14.545455,21.153846,10.714286,15.384615,13.461538,15.094340,12.244898,19.565217,24.0,...,24.528302,28.813559,10.000000,17.073171,28.571429,12.500000,30.0,27.5,13.725490,13.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,17.391304,20.000000,25.000000,21.428571,15.384615,26.923077,22.641509,18.367347,17.391304,6.0,...,24.528302,25.423729,23.333333,39.024390,40.000000,18.750000,30.0,32.5,21.568627,13.333333
88,17.391304,14.545455,19.230769,14.285714,10.769231,13.461538,18.867925,14.285714,15.217391,12.0,...,35.849057,22.033898,20.000000,17.073171,28.571429,14.583333,17.5,17.5,5.882353,6.666667
89,19.565217,9.090909,13.461538,7.142857,10.769231,9.615385,9.433962,10.204082,15.217391,14.0,...,30.188679,16.949153,13.333333,29.268293,40.000000,12.500000,22.5,35.0,15.686275,8.888889
90,13.043478,12.727273,11.538462,7.142857,10.769231,15.384615,13.207547,14.285714,8.695652,6.0,...,43.396226,33.898305,20.000000,19.512195,25.714286,16.666667,22.5,20.0,13.725490,6.666667


In [15]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

41.37124496329065


In [16]:
df = videoSimilarity("test-videos/test-video2-og.mp4", "test-videos/test-video2-increased-speed.mp4")
df

fps: 25
Total Number of keyframes saved: 92
fps: 30
Total Number of keyframes saved: 85


,0,1,2,3,4,5,6,7,8,9,...,75,76,77,78,79,80,81,82,83,84
0,52.083333,55.102041,52.459016,52.459016,85.483871,71.666667,49.180328,46.551724,44.642857,43.333333,...,40.384615,41.509434,43.636364,30.000000,35.294118,39.285714,31.914894,40.476190,34.090909,39.473684
1,43.750000,44.897959,50.819672,49.180328,62.903226,73.333333,45.901639,41.379310,42.857143,41.666667,...,40.384615,37.735849,36.363636,31.666667,37.254902,35.714286,27.659574,38.095238,31.818182,39.473684
2,52.083333,48.979592,50.819672,40.983607,51.612903,55.000000,62.295082,70.689655,75.000000,73.333333,...,44.230769,32.075472,36.363636,36.666667,43.137255,33.928571,42.553191,47.619048,43.181818,47.368421
3,37.500000,48.979592,39.344262,40.983607,40.322581,50.000000,60.655738,70.689655,89.285714,68.333333,...,30.769231,32.075472,34.545455,26.666667,33.333333,46.428571,27.659574,40.476190,38.636364,42.105263
4,45.833333,48.979592,45.901639,40.983607,41.935484,55.000000,54.098361,77.586207,82.142857,83.333333,...,34.615385,43.396226,36.363636,31.666667,45.098039,32.142857,36.170213,40.476190,38.636364,50.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,39.583333,40.816327,32.786885,34.426230,37.096774,38.333333,32.786885,32.758621,35.714286,35.000000,...,90.384615,66.037736,67.272727,60.000000,50.980392,32.142857,29.787234,40.476190,43.181818,44.736842
88,31.250000,30.612245,32.786885,32.786885,33.870968,33.333333,24.590164,32.758621,26.785714,33.333333,...,69.230769,88.679245,70.909091,61.666667,39.215686,33.928571,21.276596,40.476190,36.363636,42.105263
89,31.250000,30.612245,29.508197,34.426230,35.483871,36.666667,31.147541,27.586207,32.142857,31.666667,...,65.384615,58.490566,72.727273,58.333333,49.019608,32.142857,27.659574,33.333333,40.909091,34.210526
90,37.500000,34.693878,34.426230,34.426230,37.096774,40.000000,36.065574,41.379310,33.928571,35.000000,...,67.307692,60.377358,70.909091,81.666667,54.901961,44.642857,34.042553,52.380952,38.636364,50.000000


In [17]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

81.79873703475553


In [19]:
df = videoSimilarity("test-videos/test-video2-og.mp4", "test-videos/test-video2-watermark.mp4")
df

fps: 25
Total Number of keyframes saved: 92
fps: 30
Total Number of keyframes saved: 124


,0,1,2,3,4,5,6,7,8,9,...,114,115,116,117,118,119,120,121,122,123
0,54.6875,52.727273,50.819672,45.945946,50.0000,64.406780,83.076923,55.555556,45.161290,41.379310,...,37.037037,32.758621,34.482759,25.423729,35.849057,28.888889,24.528302,27.272727,23.076923,24.561404
1,51.5625,54.545455,50.819672,44.594595,48.4375,49.152542,60.000000,70.833333,53.225806,43.103448,...,40.740741,32.758621,34.482759,20.338983,28.301887,26.666667,24.528302,25.757576,18.461538,21.052632
2,48.4375,56.363636,49.180328,39.189189,40.6250,50.847458,47.692308,43.055556,43.548387,68.965517,...,40.740741,39.655172,32.758621,35.593220,30.188679,40.000000,35.849057,25.757576,32.307692,29.824561
3,40.6250,41.818182,42.622951,32.432432,40.6250,38.983051,40.000000,36.111111,33.870968,67.241379,...,37.037037,37.931034,31.034483,25.423729,26.415094,28.888889,30.188679,16.666667,23.076923,21.052632
4,48.4375,47.272727,45.901639,35.135135,45.3125,37.288136,41.538462,38.888889,43.548387,65.517241,...,40.740741,32.758621,32.758621,28.813559,30.188679,37.777778,30.188679,27.272727,27.692308,29.824561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,34.3750,40.000000,32.786885,27.027027,28.1250,33.898305,33.846154,29.166667,32.258065,29.310345,...,68.518519,55.172414,44.827586,27.118644,30.188679,35.555556,32.075472,22.727273,21.538462,19.298246
88,31.2500,36.363636,31.147541,21.621622,28.1250,32.203390,30.769231,25.000000,30.645161,20.689655,...,64.814815,53.448276,50.000000,20.338983,28.301887,28.888889,24.528302,16.666667,16.923077,19.298246
89,26.5625,30.909091,31.147541,27.027027,28.1250,30.508475,33.846154,26.388889,29.032258,29.310345,...,61.111111,50.000000,48.275862,25.423729,20.754717,31.111111,24.528302,16.666667,16.923077,17.543860
90,34.3750,38.181818,37.704918,28.378378,32.8125,33.898305,36.923077,26.388889,30.645161,32.758621,...,83.333333,65.517241,56.896552,27.118644,30.188679,31.111111,32.075472,22.727273,24.615385,19.298246


In [20]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

81.03385916888963


In [31]:
df = videoSimilarity("test-videos/test-video2-og.mp4", "test-videos/test-video2-BorderAndWatermark.mp4")
df

fps: 25
Total Number of keyframes saved: 92
fps: 30
Total Number of keyframes saved: 82


,0,1,2,3,4,5,6,7,8,9,...,72,73,74,75,76,77,78,79,80,81
0,20.754717,22.222222,19.402985,24.285714,23.809524,13.432836,23.333333,14.925373,13.846154,16.129032,...,14.0625,18.7500,12.5000,10.9375,14.545455,8.928571,13.333333,5.660377,8.620690,3.773585
1,13.207547,14.285714,11.940299,17.142857,31.746032,14.925373,16.666667,10.447761,10.769231,14.516129,...,14.0625,18.7500,15.6250,12.5000,9.090909,12.500000,13.333333,3.773585,5.172414,3.773585
2,15.094340,23.809524,19.402985,18.571429,19.047619,23.880597,26.666667,25.373134,26.153846,24.193548,...,23.4375,18.7500,14.0625,10.9375,23.636364,10.714286,20.000000,9.433962,8.620690,9.433962
3,20.754717,14.285714,19.402985,11.428571,15.873016,16.417910,28.333333,20.895522,29.230769,30.645161,...,28.1250,20.3125,18.7500,17.1875,20.000000,14.285714,22.222222,5.660377,10.344828,11.320755
4,18.867925,12.698413,14.925373,11.428571,14.285714,16.417910,25.000000,22.388060,35.384615,37.096774,...,28.1250,21.8750,20.3125,20.3125,21.818182,16.071429,22.222222,11.320755,6.896552,7.547170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,13.207547,19.047619,17.910448,15.714286,14.285714,10.447761,13.333333,8.955224,7.692308,14.516129,...,29.6875,29.6875,25.0000,17.1875,27.272727,16.071429,28.888889,15.094340,10.344828,9.433962
88,3.773585,15.873016,16.417910,14.285714,12.698413,14.925373,15.000000,7.462687,9.230769,11.290323,...,29.6875,29.6875,28.1250,20.3125,18.181818,16.071429,20.000000,13.207547,5.172414,3.773585
89,7.547170,9.523810,8.955224,5.714286,9.523810,10.447761,16.666667,5.970149,9.230769,8.064516,...,29.6875,32.8125,25.0000,20.3125,20.000000,12.500000,28.888889,11.320755,6.896552,5.660377
90,13.207547,9.523810,10.447761,8.571429,12.698413,7.462687,8.333333,7.462687,9.230769,14.516129,...,28.1250,31.2500,32.8125,23.4375,23.636364,19.642857,20.000000,5.660377,6.896552,7.547170


In [32]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

36.01680335513429


In [21]:
df = videoSimilarity("test-videos/test-video3-og.mp4", "test-videos/test-video3-rotated.mp4")
df

fps: 30
Total Number of keyframes saved: 20
fps: 30
Total Number of keyframes saved: 19


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,38.059701,15.662651,9.497207,7.692308,5.732484,6.535948,7.643312,4.864865,3.608247,3.243243,3.619910,1.941748,4.784689,2.185792,3.773585,3.092784,3.664921,3.225806,5.319149
1,79.104478,33.132530,17.318436,4.733728,3.184713,3.267974,6.369427,2.702703,4.639175,3.783784,3.619910,2.427184,3.349282,5.464481,3.773585,3.092784,3.141361,5.161290,3.191489
2,35.074627,76.506024,26.815642,5.917160,7.643312,5.228758,7.643312,3.783784,5.154639,5.945946,6.334842,6.796117,8.133971,10.382514,8.018868,6.701031,5.759162,2.580645,2.659574
3,22.388060,35.542169,69.832402,3.550296,3.184713,7.843137,5.095541,5.405405,8.247423,7.027027,6.334842,5.825243,8.133971,9.836066,4.716981,8.247423,4.188482,5.161290,5.851064
4,5.223881,4.819277,10.055866,72.189349,43.312102,27.450980,21.656051,13.513514,8.762887,7.027027,6.334842,3.883495,7.655502,2.185792,3.773585,3.608247,4.712042,5.161290,3.723404
5,5.223881,4.216867,8.379888,42.011834,79.617834,42.483660,31.847134,20.000000,11.855670,11.891892,5.882353,4.854369,4.784689,5.464481,2.358491,3.608247,3.141361,5.806452,4.255319
6,0.746269,3.012048,4.469274,24.852071,39.490446,69.281046,35.031847,23.243243,15.463918,8.648649,6.334842,3.883495,1.913876,1.639344,0.943396,0.515464,1.047120,4.516129,3.191489
7,2.985075,3.012048,6.145251,19.526627,33.757962,33.333333,70.700637,35.675676,18.556701,15.135135,10.859729,5.825243,5.741627,6.557377,1.886792,1.030928,2.094241,2.580645,2.659574
8,5.223881,8.433735,7.262570,23.668639,37.579618,41.176471,54.140127,76.216216,39.690722,26.486486,17.647059,14.563107,13.875598,14.754098,6.132075,5.670103,6.282723,4.516129,2.659574
9,11.194030,9.638554,9.497207,13.609467,15.286624,26.143791,30.573248,40.000000,79.381443,39.459459,29.411765,28.155340,24.401914,21.857923,15.094340,11.855670,12.041885,8.387097,5.851064


In [22]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

73.28799608585526


In [24]:
df = videoSimilarity("test-videos/test-video4-og.mp4", "test-videos/test-video4-rotated.mp4")
df

fps: 30
Total Number of keyframes saved: 34
fps: 30
Total Number of keyframes saved: 33


,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,75.000000,61.0,57.894737,61.403509,60.784314,10.759494,14.189189,10.370370,10.526316,13.698630,...,15.267176,13.286713,11.678832,10.317460,12.307692,11.724138,9.302326,10.077519,10.218978,9.022556
1,71.153846,80.0,58.771930,64.035088,60.784314,8.860759,10.135135,13.333333,9.210526,9.589041,...,11.450382,12.587413,13.138686,15.873016,12.307692,12.413793,10.077519,10.852713,12.408759,10.526316
2,63.461538,65.0,75.438596,53.508772,66.666667,12.658228,16.216216,14.814815,14.473684,12.328767,...,18.320611,15.384615,12.408759,18.253968,11.538462,13.793103,6.976744,13.953488,11.678832,10.526316
3,67.307692,72.0,60.526316,77.192982,72.549020,14.556962,14.864865,17.037037,14.473684,15.068493,...,18.320611,16.783217,12.408759,15.873016,12.307692,13.793103,7.751938,13.953488,16.788321,14.285714
4,58.653846,55.0,52.631579,60.526316,89.215686,12.658228,14.189189,12.592593,16.447368,10.273973,...,21.374046,12.587413,14.598540,19.841270,10.769231,9.655172,7.751938,12.403101,6.569343,6.766917
5,45.192308,42.0,42.982456,45.614035,54.901961,14.556962,13.513514,14.074074,18.421053,11.643836,...,15.267176,11.188811,11.678832,13.492063,14.615385,8.965517,6.201550,10.852713,12.408759,7.518797
6,19.230769,15.0,15.789474,17.543860,21.568627,78.481013,70.945946,65.925926,54.605263,60.273973,...,48.854962,37.762238,44.525547,43.650794,40.769231,37.931034,32.558140,31.007752,34.306569,26.315789
7,16.346154,17.0,13.157895,14.035088,19.607843,59.493671,75.675676,70.370370,61.184211,65.068493,...,46.564885,38.461538,48.905109,41.269841,42.307692,41.379310,40.310078,34.883721,32.116788,30.827068
8,14.423077,12.0,10.526316,9.649123,14.705882,53.797468,66.891892,73.333333,63.157895,58.219178,...,46.564885,34.965035,43.065693,39.682540,36.153846,33.103448,34.883721,34.883721,32.846715,26.315789
9,21.153846,15.0,14.912281,15.789474,22.549020,62.025316,75.000000,74.814815,80.263158,70.547945,...,58.778626,48.951049,59.124088,47.619048,50.000000,47.586207,49.612403,41.085271,32.846715,30.827068


In [25]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

78.86283665609318


In [26]:
df = videoSimilarity("test-videos/test-video5-og.mp4", "test-videos/test-video5-rotated.mp4")
df

fps: 25
Total Number of keyframes saved: 4
fps: 25
Total Number of keyframes saved: 4


,0,1,2,3
0,81.188119,23.651452,24.358974,25.461255
1,15.841584,73.858921,49.145299,22.878229
2,17.161716,47.717842,76.495726,28.413284
3,20.132013,24.481328,32.051282,78.966790


In [27]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

77.62738903433251


In [29]:
df = videoSimilarity("test-videos/test-video6-og.mp4", "test-videos/test-video6-rotated.mp4")
df

fps: 25
Total Number of keyframes saved: 41
fps: 25
Total Number of keyframes saved: 41


,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,83.950617,59.090909,37.301587,35.655738,21.145374,22.368421,18.614719,20.833333,17.435897,16.086957,...,6.8,7.569721,5.829596,6.018519,5.240175,9.216590,8.133971,7.929515,4.615385,3.265306
1,50.205761,85.454545,46.031746,38.114754,17.621145,22.368421,19.480519,19.907407,17.435897,13.913043,...,6.4,7.171315,6.726457,3.240741,6.113537,4.608295,6.698565,7.488987,3.846154,2.448980
2,42.386831,57.727273,78.968254,56.147541,26.872247,31.140351,25.541126,25.925926,26.666667,20.000000,...,8.4,9.561753,7.623318,6.481481,8.296943,9.677419,9.569378,7.929515,5.384615,3.673469
3,32.098765,40.454545,49.603175,77.459016,24.669604,30.263158,27.272727,25.925926,26.666667,20.869565,...,7.2,7.171315,6.726457,5.092593,5.240175,6.451613,5.741627,7.488987,3.461538,3.265306
4,19.753086,20.454545,24.603175,24.590164,83.259912,53.508772,40.259740,41.666667,36.410256,30.434783,...,9.6,7.171315,4.932735,8.333333,8.733624,8.294931,10.526316,5.286344,4.615385,3.265306
5,18.930041,21.363636,26.587302,26.229508,51.541850,83.333333,52.813853,50.462963,45.641026,33.478261,...,10.0,8.366534,9.417040,8.796296,8.296943,10.138249,11.483254,6.607930,4.230769,3.265306
6,16.872428,19.545455,24.603175,27.049180,42.290749,50.438596,81.818182,53.703704,45.641026,32.173913,...,9.6,8.366534,9.417040,10.185185,9.170306,8.755760,11.004785,6.607930,3.846154,5.306122
7,16.460905,21.363636,23.015873,27.049180,38.766520,47.368421,51.515152,82.407407,55.897436,39.565217,...,11.2,8.764940,10.762332,8.796296,10.480349,11.059908,11.483254,7.048458,5.000000,4.897959
8,13.580247,14.545455,19.047619,21.721311,37.004405,38.157895,38.095238,50.000000,83.076923,51.739130,...,9.6,10.756972,9.417040,10.185185,7.423581,11.520737,11.004785,5.726872,2.692308,3.673469
9,16.460905,16.363636,17.857143,20.491803,24.669604,32.017544,30.303030,46.759259,58.461538,80.000000,...,9.6,7.569721,9.865471,8.333333,6.550218,7.373272,10.526316,5.726872,3.076923,3.673469


In [30]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

78.94493168133103


In [9]:
df = videoSimilarity("test-videos/test-video7.mp4", "test-videos/test-video7-dup.mp4")
df

fps: 30
Total Number of keyframes saved: 12
fps: 30
Total Number of keyframes saved: 18


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,4.379562,5.970149,9.473684,20.689655,8.750,9.493671,10.967742,13.253012,12.804878,13.207547,10.666667,12.422360,14.375,15.189873,14.634146,20.754717,14.035088,18.128655
1,0.729927,2.238806,10.526316,22.988506,10.000,10.126582,12.903226,12.650602,14.634146,13.836478,12.666667,12.422360,11.250,15.189873,13.414634,22.641509,14.619883,18.713450
2,2.919708,5.970149,25.263158,32.183908,23.125,22.784810,19.354839,31.927711,26.219512,23.899371,21.333333,20.496894,21.875,24.683544,20.121951,33.962264,26.900585,26.315789
3,3.649635,5.970149,20.000000,27.586207,20.625,17.088608,20.645161,24.096386,18.292683,18.238994,16.000000,15.527950,18.125,18.354430,15.243902,23.270440,19.883041,21.052632
4,2.189781,4.477612,23.157895,27.586207,19.375,20.886076,19.354839,24.698795,19.512195,20.125786,16.666667,16.149068,17.500,20.253165,14.024390,23.899371,19.883041,18.713450
5,3.649635,3.731343,24.210526,32.183908,20.000,17.721519,18.064516,23.493976,18.292683,20.754717,18.666667,18.012422,15.625,21.518987,15.243902,22.641509,18.128655,18.128655
6,1.459854,1.492537,14.736842,19.540230,18.125,13.291139,15.483871,21.084337,16.463415,15.723270,13.333333,11.180124,13.125,16.455696,14.024390,22.012579,16.374269,15.789474
7,3.649635,6.716418,25.263158,29.885057,25.625,24.050633,24.516129,30.120482,26.219512,23.270440,21.333333,19.254658,19.375,27.215190,20.731707,32.704403,23.391813,22.222222
8,7.299270,8.208955,33.684211,37.931034,36.250,34.810127,36.129032,33.132530,31.707317,35.849057,27.333333,25.465839,25.625,31.012658,28.048780,37.735849,28.654971,30.409357
9,5.109489,8.208955,31.578947,36.781609,29.375,31.012658,27.741935,27.108434,20.731707,22.641509,16.666667,22.360248,17.500,29.113924,22.560976,29.559748,22.222222,22.807018


In [10]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

34.190360959870944


In [11]:
df = videoSimilarity("test-videos/test-video-tajmahal1.mp4", "test-videos/test-video-tajmahal2.mp4")
df

fps: 25
Total Number of keyframes saved: 11
fps: 25
Total Number of keyframes saved: 10


,0,1,2,3,4,5,6,7,8,9
0,1.652893,0.413223,0.444444,0.476190,0.537634,1.257862,0.000000,0.581395,0.000000,0.492611
1,1.239669,0.413223,0.000000,0.476190,0.537634,3.144654,0.000000,1.162791,0.000000,0.985222
2,0.413223,0.000000,0.000000,0.952381,0.000000,6.289308,0.591716,0.581395,0.000000,1.477833
3,0.413223,0.000000,0.000000,0.476190,0.000000,5.031447,0.000000,0.581395,0.000000,0.492611
4,0.413223,0.000000,0.888889,0.000000,1.075269,1.886792,0.000000,0.000000,0.000000,0.492611
5,0.413223,0.000000,0.444444,0.000000,1.075269,2.515723,0.000000,0.581395,0.000000,0.000000
6,1.239669,0.413223,1.333333,0.000000,1.075269,1.886792,0.591716,0.000000,0.000000,0.000000
7,0.826446,0.826446,0.444444,0.000000,1.612903,1.886792,0.000000,0.581395,0.000000,0.000000
8,0.826446,0.000000,0.888889,0.000000,0.000000,1.257862,0.000000,0.000000,1.162791,0.000000
9,2.066116,0.413223,1.333333,0.476190,0.537634,1.886792,3.550296,0.581395,1.162791,0.985222


In [12]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

2.804249594151423


In [15]:
df = videoSimilarity("test-videos/test-video-ptd-jk.mp4", "test-videos/test-video-ptd-jin.mp4")
df

fps: 30
Total Number of keyframes saved: 15
fps: 30
Total Number of keyframes saved: 24


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,13.483146,12.50,12.676056,16.981132,19.298246,25.490196,18.518519,18.518519,18.750000,25.000000,...,16.949153,15.384615,18.518519,19.117647,15.151515,14.666667,15.492958,20.000000,8.235294,7.81250
1,12.359551,13.75,14.084507,16.981132,17.543860,21.568627,20.370370,25.925926,25.000000,33.333333,...,15.254237,18.461538,20.370370,19.117647,18.181818,14.666667,15.492958,18.461538,7.058824,10.93750
2,16.853933,13.75,18.309859,16.981132,21.052632,27.450980,20.370370,18.518519,22.916667,33.333333,...,23.728814,16.923077,22.222222,17.647059,16.666667,14.666667,19.718310,20.000000,10.588235,8.59375
3,13.483146,10.00,12.676056,16.981132,21.052632,21.568627,22.222222,22.222222,18.750000,30.555556,...,22.033898,18.461538,25.925926,19.117647,16.666667,14.666667,16.901408,16.923077,7.058824,8.59375
4,13.483146,16.25,15.492958,16.981132,21.052632,19.607843,24.074074,22.222222,22.916667,27.777778,...,20.338983,16.923077,27.777778,22.058824,13.636364,12.000000,14.084507,16.923077,9.411765,25.00000
5,11.235955,12.50,14.084507,18.867925,19.298246,21.568627,22.222222,20.370370,25.000000,30.555556,...,16.949153,18.461538,18.518519,22.058824,16.666667,12.000000,14.084507,16.923077,8.235294,24.21875
6,12.359551,15.00,14.084507,20.754717,22.807018,23.529412,24.074074,24.074074,22.916667,30.555556,...,20.338983,18.461538,20.370370,19.117647,15.151515,12.000000,16.901408,20.000000,9.411765,23.43750
7,8.988764,11.25,12.676056,20.754717,17.543860,23.529412,16.666667,20.370370,27.083333,36.111111,...,15.254237,15.384615,18.518519,17.647059,13.636364,12.000000,14.084507,15.384615,9.411765,27.34375
8,11.235955,12.50,14.084507,18.867925,19.298246,21.568627,22.222222,20.370370,22.916667,36.111111,...,15.254237,16.923077,18.518519,20.588235,13.636364,12.000000,15.492958,18.461538,7.058824,24.21875
9,12.359551,12.50,14.084507,16.981132,19.298246,21.568627,18.518519,20.370370,22.916667,30.555556,...,16.949153,16.923077,16.666667,20.588235,15.151515,12.000000,12.676056,15.384615,9.411765,25.00000


In [16]:
max_df = df.max(axis=1)
max_df = max_df.loc[max_df>30]
if(len(max_df)>0):
    print(max_df.sum()/len(max_df))
else:
    print(df.max(axis=1).sum()/len(df))

32.22222222222222


In [17]:
df = videoSimilarity("test-videos/test-video-forest1.mp4", "test-videos/test-video-forest2.mp4")
df

fps: 59
Total Number of keyframes saved: 127
fps: 59
Total Number of keyframes saved: 0


""
0
1
2
3
4
...
122
123
124
125
